# Calibration of stereo camera
- Basically recycling Week 4 code
- link to get images: https://campuscvut-my.sharepoint.com/:f:/g/personal/maleklu6_cvut_cz/EnyU2LTN6QFOlGlts1-sSusB6t9Q6hAcSeDAQd2se_cJlQ?e=gff67m


## Parameters
* change parameters here to adjust the path and part of codes that are executed

In [190]:
import cv2
import numpy as np
import glob
from matplotlib import pyplot as plt
import time

In [191]:
rerun_calibration = False           # Already calibrated, could remain False
rerun_undistortion = False          # Undistorts images defined in path
rerun_rectification = False          # Rectify images in path
rerun_rect_uncalib = True           # Rectify images in path
rerun_depth = True                  # Create disparity map from rectified images
path_calib = 'images/calibration/original/'
path_dist_in = 'images/sample_without_occlusion/original/'
path_dist_out = 'images/sample_without_occlusion/undistorted/'
path_rect_in = 'images/sample_without_occlusion/original/'
path_rect_out = 'images/sample_without_occlusion/rectified/'
path_rect_uncalib_out = 'images/sample_without_occlusion/rectified_uncalibrated/'
path_depth_in = 'images/sample_without_occlusion/rectified/'
path_depth_out = 'images/sample_without_occlusion/depth/'


## Calibration
* Values from previous runs. Used if rerun_calib is set to false (needed in the next part)
* No need to run the calibratino again

In [192]:
K_left = np.array([[590.24505615, 0, 723.85543853], [
                  0, 700.56091309, 369.43859036], [0, 0, 1]])
K_right = np.array([[698.72259521, 0, 648.50704794], [
                   0, 698.6318967, 374.0875587], [0, 0, 1]])
dist_left = np.array([[-3.29479763e-01, 1.41779367e-01, -
                      1.15867147e-04, 2.53566722e-04, -3.10092346e-02]])
dist_right = np.array([[-3.25580109e-01, 1.39151479e-01, -
                     2.55229666e-04, 4.20203965e-04, -3.19659112e-02]])
mtx_left = np.array([[701.57106384,0,620.1497902],[0,701.53525002,369.95242304],[0,0,1]])
mtx_right = np.array([[699.26893071,0,649.01413957],[0,699.60352684,374.60780826],[0,0,1]])
roi = (0, 0, 1279, 719)
R = np.array([[0.98609477,-0.01017771,0.16587198],[0.01153576,0.99990735,-0.00722601],[-0.16578306,0.00903899,0.98612082]])
T = np.array([[-3.62890477],[ 0.00624134],[ 1.85263231]])
E = np.array([[-0.02240624,-1.85240425,0.01954185],[ 1.22526008, 0.01394609,3.88583833],[-0.04801674,-3.62850504, 0.02518724]])
F = np.array([[ 2.41550145e-07,1.68252147e-05,-6.51507831e-03], [-1.32106155e-05,-1.26687471e-07,-1.51199226e-02],[5.14696937e-03,1.21641825e-02,1.00000000e+00]])

### Function

In [193]:
def calibration(debug):

    # Write the image names
    images_left = glob.glob(path_calib+'left-*.png')
    images_right = glob.glob(path_calib+'right-*.png')
    assert images_left
    assert images_right

    # Implement the number of vertical and horizontal corners
    nb_vertical = 9
    nb_horizontal = 6

    # Prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((nb_horizontal*nb_vertical, 3), np.float32)
    objp[:, :2] = np.mgrid[0:nb_vertical, 0:nb_horizontal].T.reshape(-1, 2)

    # Arrays to store object points and image points from all the images.
    objpoints = []  # 3d point in real world space
    imgpoints_left = []  # 2d points in image plane.
    imgpoints_right = []  # 2d points in image plane.

    for i in range(0,len(images_left)):

        # Load the image
        gray_left = cv2.cvtColor(cv2.imread(images_left[i]), cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(cv2.imread(images_right[i]), cv2.COLOR_BGR2GRAY)

        # Implement findChessboardCorners here
        ret_left, corners_left = cv2.findChessboardCorners(gray_left, (nb_vertical, nb_horizontal))
        ret_right, corners_right = cv2.findChessboardCorners(gray_right, (nb_vertical, nb_horizontal))

        # If found, add object points, image points (after refining them)
        if ret_left == True and ret_right==True:
            objpoints.append(objp)
            imgpoints_left.append(corners_left)
            imgpoints_right.append(corners_right)

    # Get the camera matrix
    _, mtx_left, dist_left, _, _ = cv2.calibrateCamera(objpoints, imgpoints_left, gray_left.shape[::-1], None, None)
    _, mtx_right, dist_right, _, _ = cv2.calibrateCamera(objpoints, imgpoints_right, gray_right.shape[::-1], None, None)
    K_left, roi_left = cv2.getOptimalNewCameraMatrix(mtx_left, dist_left, gray_left.shape[::-1], alpha=0)
    K_right, roi_right = cv2.getOptimalNewCameraMatrix(mtx_right, dist_right, gray_left.shape[::-1], alpha=0)
    _, K_left, dist_left, K_right, dist_right, R, T, E, F = cv2.stereoCalibrate(objpoints, imgpoints_left, imgpoints_right, K_left, dist_left, K_right, dist_right, gray_left.shape[::-1])

    # Print calibrated values
    if debug:
        print('K_left', K_left)
        print('K_right',K_right)
        print('dist_left',dist_left)
        print('dist_right',dist_right)
        print('mtx_left',mtx_left)
        print('mtx_right',mtx_right)
        print('roi - same for both',roi)
        print('R',R)
        print('T',T)
        print('E',E)
        print('F',F)

    return K_left, K_right, dist_left, dist_right, mtx_left, mtx_right, roi, objpoints, imgpoints_left, imgpoints_right, R, T, E, F

### Program

In [194]:
if rerun_calibration:
    K_left, K_right, dist_left, dist_right, mtx_left, mtx_right, roi, objpoints, imgpoints_left, imgpoints_right, R, T, E, F = calibration(debug=False)


## UNDISTORTION

### Function

In [195]:
def undistort(side,mtx,dist,K,roi):
    if side == 'left':
        path_in = path_dist_in+'left/*.png'
        path_out = path_dist_out+'left/left-'
    elif side == 'right':
        path_in = path_dist_in+'right/*.png'
        path_out = path_dist_out+'right/right-'

    images = glob.glob(path_in)
    assert images

    i = 0
    for fname in images:
        # undistort
        img = cv2.imread(fname)
        dst = cv2.undistort(img, mtx, dist, None, K)

        # crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        # save image
        cv2.imwrite(path_out+str(i)+'.png', dst)
        i += 1

    print("Undistortion "+side+ " done")

### Program

In [196]:
if rerun_undistortion:
    undistort('left',mtx_left,dist_left,K_left, roi)
    undistort('right',mtx_right,dist_right,K_right, roi)

## Rectification

### Functions


In [197]:
def draw_lines(img1, img2, lines, pts1, pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
    (r, c) = img1.shape
    img1 = cv2.cvtColor(img1, cv2.COLOR_GRAY2BGR)
    img2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
    for r, pt1, pt2 in zip(lines, pts1, pts2):
        color = tuple(np.random.randint(0, 255, 3).tolist())
        x0, y0 = map(int, [0, -r[2]/r[1]])
        x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
        img1 = cv2.line(img1, (x0, y0), (x1, y1), color, 2)
        img1 = cv2.circle(img1, tuple(pt1), 5, color, -1)
        img2 = cv2.circle(img2, tuple(pt2), 5, color, -1)
    return img1, img2

def draw_epipolar_lines(gray_left, gray_right):
    '''Draws epipolar lines to the image'''
    # Create a sift detector
    sift = cv2.SIFT_create()

    # Find the keypoints and descriptors with SIFT
    kp_left, des_left = sift.detectAndCompute(gray_left, None)
    kp_right, des_right = sift.detectAndCompute(gray_right, None)

    # Match points
    matches = cv2.BFMatcher().match(des_left, des_right)
    matches = sorted(matches, key=lambda x: x.distance)
    nb_matches = 200  # Using 200 best matches
    good = []
    pts1 = []
    pts2 = []
    for m in matches[:nb_matches]:
        good.append(m)
        pts1.append(kp_left[m.queryIdx].pt)
        pts2.append(kp_right[m.trainIdx].pt)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)

    # Get fundamental matrix
    F, inliers = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC)

    # Remove outliers
    pts1 = pts1[inliers.ravel() == 1]
    pts2 = pts2[inliers.ravel() == 1]

    # Draw lines
    lines1 = cv2.computeCorrespondEpilines(pts2.reshape(-1, 1, 2), 2, F)
    lines1 = lines1.reshape(-1, 3)
    epilines_left, keypoints_left = draw_lines(gray_left, gray_right, lines1, pts1, pts2)
    lines2 = cv2.computeCorrespondEpilines(pts1.reshape(-1, 1, 2), 1, F)
    lines2 = lines2.reshape(-1, 3)
    epilines_right, keypoints_right = draw_lines(gray_right, gray_left, lines2, pts2, pts1)

    fig, axs = plt.subplots(1, 2, constrained_layout=True, figsize=(10, 10))
    axs[0].imshow(epilines_left)
    axs[0].set_title('left epipolar lines')
    axs[1].imshow(epilines_right)
    axs[1].set_title('right epipolar lines')
    plt.show()

def rectify(img_left, img_right, K_left, dist_left, K_right, dist_right, R, T, E, F,debug=False):
    '''
    Function that we used in project week 4, but the rectification is incorrect
    '''

    # Change to grayscale
    
    #F,_,_= draw_epipolar_lines(gray_left, gray_right, debug)

    # Find projection essential matrix E
    #E = K_left.T@F@K_right
    #R_right, _, t_right = cv2.decomposeEssentialMat(E)
    # R_left = np.identity(3)
    # t_left = np.zeros(shape=(3,1))
    # P_left = np.hstack((K_left@R_left, K_left@t_left))
    # P_right = np.hstack((K_right@R_right, K_right@t_right))

    
    R_left, R_right, P_left, P_right, Q, roi_left, roi_right = cv2.stereoRectify(K_left, dist_left, K_right, dist_right, gray_left.shape[::-1], R,T,flags=cv2.CALIB_ZERO_DISPARITY,alpha=0)
    #TODO: Look at Q: disparity to depth matching matrix
    
    # Rectify images
    leftMapX, leftMapY = cv2.initUndistortRectifyMap(K_left, dist_left, R_left, P_left, gray_left.shape[::-1], cv2.CV_32FC1)
    left_rectified = cv2.remap(gray_left, leftMapX, leftMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)
    rightMapX, rightMapY = cv2.initUndistortRectifyMap(K_right, dist_right, R_right, P_right, gray_left.shape[::-1], cv2.CV_32FC1)
    right_rectified = cv2.remap(gray_right, rightMapX, rightMapY, cv2.INTER_LINEAR, cv2.BORDER_CONSTANT)

    if debug:
        draw_epipolar_lines(left_rectified,right_rectified)

    return left_rectified, right_rectified

def rectify_uncalibrated(img_left, img_right, debug=False):

    # Find the keypoints and descriptors with SIFT
    kp_left, des_left = sift.detectAndCompute(img_left, None)
    kp_right, des_right = sift.detectAndCompute(img_right, None)

    # Match points
    matches = cv2.BFMatcher().match(des_left, des_right)
    matches = sorted(matches, key=lambda x: x.distance)
    nb_matches = 200  # Using 200 best matches
    good = []
    pts1 = []
    pts2 = []
    for m in matches[:nb_matches]:
        good.append(m)
        pts1.append(kp_left[m.queryIdx].pt)
        pts2.append(kp_right[m.trainIdx].pt)
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)

    # Get fundamental matrix
    F, inliers = cv2.findFundamentalMat(pts1, pts2, method=cv2.FM_RANSAC)

    h1, w1 = img_left.shape
    h2, w2 = img_right.shape
    _, H1, H2 = cv2.stereoRectifyUncalibrated(np.float32(pts1), np.float32(pts2), F, imgSize=(w1, h1))
    
    # Rectify images and save them
    left_rectified = cv2.warpPerspective(img_left, H1, (w1, h1))
    right_rectified = cv2.warpPerspective(img_right, H2, (w2, h2))

    if debug:
        draw_epipolar_lines(left_rectified,right_rectified)

    return left_rectified,right_rectified

### Rectification

In [198]:
if rerun_rectification:

    # Read the undistorted images
    imagesL = glob.glob(path_rect_in+'left/*.png')
    imagesR = glob.glob(path_rect_in+'right/*.png')
    assert imagesL
    assert imagesR

    # Create a sift detector
    sift = cv2.SIFT_create()

    for i in range(0, len(imagesL)):
        gray_left = cv2.cvtColor(cv2.imread(imagesL[i]), cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(cv2.imread(imagesR[i]), cv2.COLOR_BGR2GRAY)
        left_rectified, right_rectified = rectify(gray_left, gray_right, K_left, dist_left, K_right, dist_right, R, T, E, F, debug=False) 

        # Save images into folder
        cv2.imwrite(path_rect_out+'left/left-'+str(i)+'.png', left_rectified)
        cv2.imwrite(path_rect_out+'right/right-'+str(i)+'.png', right_rectified)
    print("Calibrated Rectification done")

if rerun_rect_uncalib:

    # Read the undistorted images
    imagesL = glob.glob(path_rect_in+'left/*.png')
    imagesR = glob.glob(path_rect_in+'right/*.png')
    assert imagesL
    assert imagesR

    # Create a sift detector
    sift = cv2.SIFT_create()

    for i in range(0, len(imagesL)):
        gray_left = cv2.cvtColor(cv2.imread(imagesL[i]), cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(cv2.imread(imagesR[i]), cv2.COLOR_BGR2GRAY)
        left_rectified, right_rectified = rectify_uncalibrated(gray_left, gray_right, debug=False) 

        # Save images into folder
        cv2.imwrite(path_rect_uncalib_out+'left/left-'+str(i)+'.png', left_rectified)
        cv2.imwrite(path_rect_uncalib_out+'right/right-'+str(i)+'.png', right_rectified)
    print("Uncalibrated Rectification done")


Uncalibrated Rectification done


## Image Depth

### Function


In [199]:
def depth_map(debug):

    # Read the undistorted images
    imagesL = glob.glob(path_depth_in+'left/*.png')
    imagesR = glob.glob(path_depth_in+'right/*.png')
    assert imagesL
    assert imagesR

    # Iterate through the images
    for i in range(0, len(imagesL)):
        img_left = cv2.imread(path_depth_in+'left/left-'+str(i)+'.png')
        img_right = cv2.imread(path_depth_in+'right/right-'+str(i)+'.png')
        gray_left = cv2.cvtColor(img_left, cv2.COLOR_BGR2GRAY)
        gray_right = cv2.cvtColor(img_right, cv2.COLOR_BGR2GRAY)

        if debug:
            f, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(18, 18))
            ax_left.imshow(gray_left,cmap='gray')
            ax_right.imshow(gray_right,cmap='gray')
            plt.show()

        min_disp = 7  # 7
        num_disp = 3*16  # 3*16
        block_size = 9  # 5, 11
        stereo = cv2.StereoBM_create(numDisparities=num_disp, blockSize=block_size)
        stereo.setMinDisparity(min_disp)
        stereo.setDisp12MaxDiff(200)  # 200
        stereo.setUniquenessRatio(1)  # 1
        stereo.setSpeckleRange(10)  # 3
        stereo.setSpeckleWindowSize(1)  # 3
        disp = stereo.compute(gray_left, gray_right).astype(np.float32) / 16.0

        # Save image
        cv2.imwrite(path_depth_out+'img-'+str(i)+'.png', disp)

        if debug:
            f, (ax_left, ax_middle, ax_right) = plt.subplots(1, 3, figsize=(18, 18))
            ax_left.imshow(gray_left,cmap='gray')
            ax_middle.imshow(gray_right,cmap='gray')
            ax_right.imshow(disp,cmap='gray')
            plt.show()

### Program


In [200]:
if rerun_depth:
    depth_map(debug=False)